In [6]:
! pip install selenium

In [56]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os
import praw
import pandas as pd
from dotenv import load_dotenv

import time

In [49]:
# Set message title
message_subject = 'Request access for academic research'
# Get the message
with open("private_subreddits_contact_letter.txt","r") as file:
    message_text=file.read()
message_text

"Hello, \n\nMy name is Andreea, and I am a data science master's student at Birkbeck College, University of London. My thesis research focuses on prevalent discussion themes within autism Reddit communities. In particular, I'm planning to track the frequency of specific keywords linked to Applied Behavioural Analysis and assess the overall sentiment of discussions revolving around this topic. My aim is to gain a comprehensive understanding of how the autism community perceives Applied Behavioural Analysis. I am reaching out to respectfully request permission to join and access the discussions in your private subreddit. \n\nI understand how crucial privacy and security are in your community, and thus I am committed to maintaining the anonymity and confidentiality of all members. The information gathered will be used strictly for academic purposes. \n\nIn recognition of the trust and sensitivity associated with your community's discussions, I am more than willing to adhere to any guideli

In [50]:
load_dotenv()
# login credentials
REDDIT_USERNAME = os.getenv('REDDIT_USERNAME')
REDDIT_PASSWORD = os.getenv('REDDIT_PASSWORD')


In [57]:
# Get all private subreddits
asd_private_subreddits = pd.read_csv('asd_new_null_rows_list.csv')
asd_private_subreddits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             36 non-null     int64  
 1   restrict_posting       0 non-null      float64
 2   display_name           36 non-null     object 
 3   title                  36 non-null     object 
 4   display_name_prefixed  36 non-null     object 
 5   subscribers            0 non-null      float64
 6   name                   36 non-null     object 
 7   public_description     26 non-null     object 
 8   community_reviewed     0 non-null      float64
 9   created                36 non-null     float64
 10  subreddit_type         36 non-null     object 
 11  id                     36 non-null     object 
 12  over18                 0 non-null      float64
 13  header_title           0 non-null      float64
 14  description            0 non-null      float64
 15  url     

In [58]:
asd_private_subreddits[['display_name', 'url']]

,display_name,url
0,neurodivergents_ZA,/r/neurodivergents_ZA/
1,LateStageAutism,/r/LateStageAutism/
2,asd,/r/asd/
3,AutisticWomen,/r/AutisticWomen/
4,HighFunctioning,/r/HighFunctioning/
5,AutismArtistic,/r/AutismArtistic/
6,autismspectrum,/r/autismspectrum/
7,AspiePartners,/r/AspiePartners/
8,autismus_treff,/r/autismus_treff/
9,bottomlulz,/r/bottomlulz/


In [94]:
# Start a browser session - browser used = Chrome
driver = webdriver.Chrome() 

In [95]:
# Open Reddit login page
driver.get("https://www.reddit.com/login")

In [96]:
# Locate the username and password fields and enter credentials
username_field = driver.find_element(By.XPATH,"//*[@id='loginUsername']")
password_field = driver.find_element(By.XPATH,"//*[@id='loginPassword']")

username_field.send_keys(REDDIT_USERNAME)
password_field.send_keys(REDDIT_PASSWORD)

# Submit the login form
password_field.send_keys(Keys.RETURN)
# Wait for a few seconds to allow the login process to complete
time.sleep(5)

In [97]:
# Reject cookies
reject_cookies_link = driver.find_element(By.XPATH, "//button[contains(text(), 'Reject non-essential')]")
reject_cookies_link.click()
time.sleep(5)

In [99]:
# count is only used for testing - removed when sending the message to allow all 36 tabs to be opened
count = 0
for sub in asd_private_subreddits['url']:
    # Navigating to a specific subreddit URL
    subreddit_url = f"https://www.reddit.com/{sub}"
    driver.execute_script("window.open('" + subreddit_url + "','_blank');")
    time.sleep(10)
    
    # Switch to the newly opened tab
    driver.switch_to.window(driver.window_handles[-1])
    # Wait for a few seconds to let the new tab load
    time.sleep(5)

    # Find and click on the "Message Mods" link using the specified XPath
    message_mods_link = driver.find_element(By.XPATH, "//a[contains(text(), 'message the moderators')]")
    message_mods_link.click()
    # Wait for a few seconds to let the new tab load
    time.sleep(5)

    
    # Add the subject to the subject input field
    # Find the iframe element by its class attribute
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "saPujbGMyXRwqISHcmJH9"))
    )
    # Switch to the iframe context
    driver.switch_to.frame(iframe)

    # Find the <input> element within the iframe by its name attribute
    subject_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "subject"))
    )
    # Clear any existing text and type the new text
    subject_input.clear()
    subject_input.send_keys(message_subject)
    # Wait for a few seconds to allow the text to be typed
    time.sleep(3)


    # Add the message to the message field
    # Find the <textarea> element within the iframe by its name attribute
    message_textarea = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/form[2]/div[7]/div/div/div[2]/div/div[1]/textarea"))
    )
    # Clear any existing text and type the new text
    message_textarea.clear()
    message_textarea.send_keys(message_text)

    # Switch back to the main content
    driver.switch_to.default_content()

    # Wait for a few seconds to allow the text to be typed
    time.sleep(3)

    count += 1
    if count > 1:
        break
